In [1]:
import pandas as pd
import requests
import json
from datetime import date
import numpy as np

API_KEY = "KEY"

In [2]:
REP_NUM = 3
COMPOUND_LAYOUT = 1
TX_PLATE_BARCODE = "12345"

CELL_TYPE = "52b2636a-e77c-4574-b6f2-78ad283c489e" # -> hard-coded
CULTURE_STATE = "Active" # -> hard-coded
PASSAGE = 8
PARENT_CULTURE = "CUL053"

NUM_CELLS_SEEDED = 400_000
VOL = 0.17

TX_PROTOCOL_ID = "PR003-V1"
PREP_BY = "Yang Gao"

In [3]:
treatment_info = pd.read_csv("../../data/Treatment_Layout_for_testing.csv")
treatment_info

,Compound ID,Compound Aliquot ID,Dose,Dose Unit,Condition,Plate Well
0,TAL501,TAL501-CMS101,20.0,uM,Test compound,C7
1,TAL502,TAL502-CMS102,20.0,uM,Test compound,F9
2,TAL503,TAL503-CMS103,20.0,uM,Test compound,F5
3,TAL504,TAL504-CMS104,20.0,uM,Test compound,H10
4,TAL505,TAL505-CMS105,20.0,uM,Test compound,H1
...,...,...,...,...,...,...
87,dBET6,CMS201,50.0,nM,Positive control,B8
88,NaN,NaN,NaN,NaN,DMSO,A4
89,NaN,NaN,NaN,NaN,DMSO,E3
90,NaN,NaN,NaN,NaN,DMSO,B4


In [4]:
def find_row_by_id(manager, id):
    session = requests.Session()
    api_call = 'https://api.scispot.io/v2/labsheets/find-row-by-id'
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "id": id
    }
    ret = session.post(api_call, json=payload)
    return json.loads(ret.text)

In [5]:
# extract info
def define_compound_inputs(df_row):
    if df_row["Condition"] == "DMSO":
        name = "THP1_" + "DMSO" + "_rep" + str(REP_NUM)
        tx_stock_id = ""
        tx_dose = ""
        tx_dose_unit = ""
    else:
        name = "THP1_" + df_row["Compound ID"] + "_rep" + str(REP_NUM)
        tx_stock_id = find_row_by_id("Compound Stock", df_row["Compound Aliquot ID"])['rows'][0][0]# df_row["Compound Aliquot ID"]
        tx_dose = df_row["Dose"]
        tx_dose_unit = df_row["Dose Unit"]

    tx_type = df_row["Condition"]
    
    return cell_culture_rows(name, tx_type, tx_stock_id, tx_dose, tx_dose_unit)

In [6]:
def cell_culture_rows(name, tx_type, tx_stock_id, tx_dose, tx_dose_unit):
    rows = [
    "AUTOMATIC", # Registry ID - don't do anything with   Registry ID,
    CELL_TYPE, # Cell Type - HARD CODED - THP1       Cell Type,
    name, # Name (temporary field)                   Name,
    CULTURE_STATE, # Culture State                   Culture State,
    "", # Stock Type                                 Stock Type,
    "", # Mycoplasma Test                            Mycoplasma Test,
    PASSAGE, # Passage                               Passage,
    "", # Supplier                                   Supplier,
    "", # Supplier Batch ID                          Supplier Batch ID,
    "", # Received Date                              Received Date,
    PARENT_CULTURE, # Parent Culture                 Parent Culture,
    NUM_CELLS_SEEDED, # Number of Cells Seeded       Number of Cells Seeded,
    VOL, # Volume (mL)                               Volume (mL),
    tx_type, # Treatment Type                        Treatment Type,
    tx_stock_id, # Treatment Compound Stock ID       Treatment Compound Stock ID,
    tx_dose, # Treatment Dose                        Treatment Dose,
    tx_dose_unit, # Treatment Dose Unit              Treatment Dose Unit,
    "", # Child Cultures                             Child Cultures,
    "", # Freezing Protocol ID                       Freezing Protocol ID,
    "", # Culture Protocol ID                        Culture Protocol ID,
    "", # Treatment Protcol ID           Treatment Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    ""# Storage Location                            Storage Location
    ]
    return rows
#
#

In [7]:
def add_rows(manager, rows):
    session = requests.Session()
    api_call = 'https://api.scispot.io/v2/labsheets/add-rows'
    payload = {
        "apiKey": API_KEY,
        "manager": manager,
        "rows": [
            rows
        ]
    }
    ret = session.post(api_call, json=payload)
    print(ret.text)
    return json.loads(ret.text)

In [8]:
manifest_list = []

In [9]:
for index, row in treatment_info.iterrows():
    row_input = define_compound_inputs(row)
    print(row_input)
    res = add_rows("Cell Culture", row_input)
    manifest_list.append({"well": row["Plate Well"], "name": res[0]["uuid"]})
    print()

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL501_rep3', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', 'cab6c280-59e6-45a4-be59-e3255a0ae872', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-27', '', '']
[{"row":"1","success":"true","uuid":"0f043b72-a461-495f-b80b-efe8043c46bb"}]

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL502_rep3', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', 'b5f4c926-7532-4561-9ead-ba3014cbecf6', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-27', '', '']
[{"row":"1","success":"true","uuid":"0b1b3583-e6a8-459a-951c-d2b3e6357683"}]

['AUTOMATIC', '52b2636a-e77c-4574-b6f2-78ad283c489e', 'THP1_TAL503_rep3', 'Active', '', '', 8, '', '', '', 'CUL053', 400000, 0.17, 'Test compound', '51ffb568-9427-4c6a-8cfe-8f2873afb2a2', 20.0, 'uM', '', '', '', '', 'Yang Gao', '2023-07-27', '', '']
[{"row":"1","success":"true","uuid":"6b289d46-9ea5-46ab-87a9-72da2ae5174f"}]

['AUTOMATIC', '5

In [10]:
print(manifest_list)

[{'well': 'C7', 'name': '0f043b72-a461-495f-b80b-efe8043c46bb'}, {'well': 'F9', 'name': '0b1b3583-e6a8-459a-951c-d2b3e6357683'}, {'well': 'F5', 'name': '6b289d46-9ea5-46ab-87a9-72da2ae5174f'}, {'well': 'H10', 'name': '801901bc-7d05-4998-a159-3f0b835b2243'}, {'well': 'H1', 'name': 'dd6df77d-7741-4720-8005-c43c112c05bb'}, {'well': 'C8', 'name': '035949b4-0713-4886-8cb5-622cad4a9a20'}, {'well': 'E1', 'name': 'a994e7d0-abe3-40bd-a990-f68939ca32dc'}, {'well': 'F10', 'name': '08958767-1171-4a69-ba80-49747ad083a9'}, {'well': 'G12', 'name': '2084b31d-60fe-4346-be85-b5fdb0ebcaee'}, {'well': 'H7', 'name': 'a5d82e69-7b22-4692-b8e7-d11cd9b69dab'}, {'well': 'A7', 'name': '4a15e157-15d9-4424-94e5-d5c2202672af'}, {'well': 'B6', 'name': 'd34820eb-8423-4aca-804a-650f899bed4e'}, {'well': 'H9', 'name': '7cbfea72-88d9-49cd-8ba2-1c916c24e104'}, {'well': 'F2', 'name': 'd122babc-d597-4032-ada9-63032f072d34'}, {'well': 'E11', 'name': '5408d944-f86c-48dc-91d7-2c6726518c8b'}, {'well': 'E2', 'name': 'a7ac8805-c6

In [16]:
def make_new_plate(manager, well_info, manifest_name):
    url = "https://api.scispot.io/tryingtofixcors/manifest/create"
    payload = {
        "apiKey": API_KEY,
        "name": "TEST SAM",
                "template": "96-well plate",
         "plates": [
            {
                "template": "96-well plate",
                "wells": 96,
                "labsheets": [
                    {
                        "labsheet": manager,
                        "items": well_info
                    }
                ]
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text)
    
# make_new_plate()

In [13]:
# ## make manifest
manifest_name = TX_PLATE_BARCODE + "_Set" + str(COMPOUND_LAYOUT) + "_Rep" + str(REP_NUM) + "_Treatment"
print(manifest_name)

12345_Set1_Rep3_Treatment


In [17]:
make_new_plate("Cell Culture", manifest_list, manifest_name)

{"success":false,"message":"invalid labsheet data"}
